Pokemon PCA

In [198]:
#from dash import Dash, html, dcc, Input, Output
#import dash_cytoscape as cyto

import networkx as nx

import numpy as np
import pandas as pd
import yaml
import os


In [199]:
with open(os.path.join('data', 'pokemon-forms.yaml'), 'r') as file:
    data = yaml.safe_load(file)

pokemon_forms = pd.DataFrame(data).T


In [200]:
pokemon_forms.head()

,pokemonid,formid,formname,gen,release,type1,type2,stats,species,height,weight,gender,catch-rate,base-exp,egg-cycles,friendship,growth-rate,ev-yield
bulbasaur,bulbasaur,None,None,1,red-blue,grass,poison,"{'hp': 45, 'attack': 49, 'defense': 49, 'spatk...",Seed,0.7,6.9,1:7,45,64,20,50,medium slow,{'spatk': 1}
ivysaur,ivysaur,None,None,1,red-blue,grass,poison,"{'hp': 60, 'attack': 62, 'defense': 63, 'spatk...",Seed,1,13,1:7,45,142,20,50,medium slow,"{'spatk': 1, 'spdef': 1}"
venusaur,venusaur,None,None,1,red-blue,grass,poison,"{'hp': 80, 'attack': 82, 'defense': 83, 'spatk...",Seed,2,100,1:7,45,236,20,50,medium slow,"{'spatk': 2, 'spdef': 1}"
venusaur-mega,venusaur,mega,Mega Venusaur,6,x-y,grass,poison,"{'hp': 80, 'attack': 100, 'defense': 123, 'spa...",Seed,2.4,155.5,1:7,45,281,20,50,medium slow,"{'spatk': 2, 'spdef': 1}"
charmander,charmander,None,None,1,red-blue,fire,None,"{'hp': 39, 'attack': 52, 'defense': 43, 'spatk...",Lizard,0.6,8.5,1:7,45,62,20,50,medium slow,{'speed': 1}


In [201]:
pokemon_forms.columns

Index(['pokemonid', 'formid', 'formname', 'gen', 'release', 'type1', 'type2',
       'stats', 'species', 'height', 'weight', 'gender', 'catch-rate',
       'base-exp', 'egg-cycles', 'friendship', 'growth-rate', 'ev-yield'],
      dtype='object')

In [202]:
cols = []
for col in pokemon_forms.columns:
    try : 
        
        if len(pokemon_forms[col].unique()) < 20 or isinstance(pokemon_forms[col].unique()[0], (float, int)): 
            print(f"{col} : {pokemon_forms[col].unique()}\n")
            cols.append(col)
    except TypeError as TE : 
        pass 


gen : [1 6 7 8 9 2 3 4 5]

release : ['red-blue' 'x-y' 'omega-ruby-alpha-sapphire' 'sun-moon'
 'lets-go-pikachu-eevee' 'sword-shield' 'legends-arceus' 'scarlet-violet'
 'gold-silver' 'ruby-sapphire' 'diamond-pearl' 'black-white'
 'ultra-sun-ultra-moon']

type1 : ['grass' 'fire' 'water' 'bug' 'normal' 'dark' 'poison' 'electric' 'ground'
 'ice' 'fairy' 'steel' 'fighting' 'psychic' 'rock' 'ghost' 'dragon'
 'flying']

type2 : ['poison' None 'flying' 'dragon' 'normal' 'psychic' 'steel' 'ground'
 'fairy' 'grass' 'rock' 'fighting' 'electric' 'ice' 'dark' 'ghost' 'fire'
 'water' 'bug']

height : [0.7 1 2 2.4 0.6 1.1 1.7 0.5 1.6 0.3 1.4 1.5 2.2 1.2 3.5 0.4 0.8 1.3 0.9
 0.2 1.9 1.8 8.8 10.9 3 6.5 2.5 2.1 4 2.3 9.2 10.5 5.2 3.8 14.5 2.7 6.2
 4.5 9.8 5 7 10.8 5.4 4.2 6.3 3.7 6.9 3.2 3.3 0.1 2.6 2.8 2.9 3.6 5.8 8.2
 3.9 3.4 5.5 7.5 20 100 12]

weight : [6.9 13 100 155.5 8.5 19 90.5 110.5 100.5 9 22.5 85.5 101.1 2.9 9.9 32 3.2
 10 29.5 40.5 1.8 30 39.5 50.5 3.5 3.8 18.5 25.5 2 38 65 6 21 12 40 55 7


In [203]:
cols

['gen',
 'release',
 'type1',
 'type2',
 'height',
 'weight',
 'gender',
 'catch-rate',
 'base-exp',
 'egg-cycles',
 'friendship',
 'growth-rate']

In [204]:
stats = pd.DataFrame()
for row in pokemon_forms.index:
    new_line = pd.DataFrame(list(pokemon_forms.loc[row, "stats"].values()), 
                        index=pokemon_forms.loc[row, "stats"].keys(), 
                        columns=[row]).T
    stats = pd.concat([stats, new_line])


In [205]:
pokemon_forms = pd.merge(pokemon_forms, stats, left_index=True, right_index=True)
pokemon_forms = pokemon_forms.drop("stats", axis = 1)


In [206]:
pokemon_forms.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1215 entries, bulbasaur to pecharunt
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pokemonid    1215 non-null   object
 1   formid       230 non-null    object
 2   formname     230 non-null    object
 3   gen          1215 non-null   object
 4   release      1215 non-null   object
 5   type1        1215 non-null   object
 6   type2        669 non-null    object
 7   species      1214 non-null   object
 8   height       1215 non-null   object
 9   weight       1214 non-null   object
 10  gender       1214 non-null   object
 11  catch-rate   1214 non-null   object
 12  base-exp     1192 non-null   object
 13  egg-cycles   1192 non-null   object
 14  friendship   1192 non-null   object
 15  growth-rate  1215 non-null   object
 16  ev-yield     1215 non-null   object
 17  hp           1215 non-null   int64 
 18  attack       1215 non-null   int64 
 19  defense      1215 n

In [207]:
pokemon_forms["gen"] = pokemon_forms["gen"].astype("int64")
num = ["height", "weight", "catch-rate", "base-exp", "egg-cycles", "friendship" ]
for col in num : 
    pokemon_forms[col] = pokemon_forms[col].replace('None', np.NaN)
    #pokemon_forms[col] = pokemon_forms[col].astype("int64")
    print(f"{col} has {pokemon_forms[col].isna().sum()} None values")
    pokemon_forms[col].fillna(np.nan)
    pokemon_forms[col] = pd.to_numeric(pokemon_forms[col])



height has 0 None values
weight has 1 None values
catch-rate has 1 None values
base-exp has 23 None values
egg-cycles has 23 None values
friendship has 23 None values


/tmp/ipykernel_48421/3526015298.py:7: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [208]:
catch_rate_index = pokemon_forms[pokemon_forms['catch-rate'] == 255].index
print(catch_rate_index)

Index(['caterpie', 'weedle', 'pidgey', 'rattata', 'rattata-alolan', 'spearow',
       'ekans', 'sandshrew', 'sandshrew-alolan', 'zubat', 'oddish', 'diglett',
       'diglett-alolan', 'meowth', 'meowth-alolan', 'meowth-galarian',
       'poliwag', 'bellsprout', 'geodude', 'geodude-alolan', 'magikarp',
       'sentret', 'hoothoot', 'ledyba', 'spinarak', 'hoppip', 'wooper',
       'wooper-paldean', 'poochyena', 'zigzagoon', 'zigzagoon-galarian',
       'wurmple', 'lotad', 'seedot', 'shroomish', 'slakoth', 'nincada',
       'nosepass', 'skitty', 'numel', 'spoink', 'spinda', 'trapinch', 'swablu',
       'baltoy', 'feebas', 'spheal', 'clamperl', 'starly', 'bidoof',
       'kricketot', 'budew', 'bronzor', 'bonsly', 'patrat', 'watchog',
       'lillipup', 'purrloin', 'pidove', 'roggenrola', 'audino', 'audino-mega',
       'tympole', 'sewaddle', 'venipede', 'maractus', 'minccino', 'vanillite',
       'ferroseed', 'elgyem', 'bunnelby', 'fletchling', 'scatterbug',
       'pikipek', 'yungoos', 'gr

In [209]:
pokemon_forms.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1215 entries, bulbasaur to pecharunt
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pokemonid    1215 non-null   object 
 1   formid       230 non-null    object 
 2   formname     230 non-null    object 
 3   gen          1215 non-null   int64  
 4   release      1215 non-null   object 
 5   type1        1215 non-null   object 
 6   type2        669 non-null    object 
 7   species      1214 non-null   object 
 8   height       1215 non-null   float64
 9   weight       1214 non-null   float64
 10  gender       1214 non-null   object 
 11  catch-rate   1214 non-null   float64
 12  base-exp     1192 non-null   float64
 13  egg-cycles   1192 non-null   float64
 14  friendship   1192 non-null   float64
 15  growth-rate  1215 non-null   object 
 16  ev-yield     1215 non-null   object 
 17  hp           1215 non-null   int64  
 18  attack       1215 non-null   int64  
 19

In [210]:
poke_num = pokemon_forms.select_dtypes(include="number")
poke_num = poke_num.dropna()
poke_num.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1191 entries, bulbasaur to miraidon
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   gen         1191 non-null   int64  
 1   height      1191 non-null   float64
 2   weight      1191 non-null   float64
 3   catch-rate  1191 non-null   float64
 4   base-exp    1191 non-null   float64
 5   egg-cycles  1191 non-null   float64
 6   friendship  1191 non-null   float64
 7   hp          1191 non-null   int64  
 8   attack      1191 non-null   int64  
 9   defense     1191 non-null   int64  
 10  spatk       1191 non-null   int64  
 11  spdef       1191 non-null   int64  
 12  speed       1191 non-null   int64  
dtypes: float64(6), int64(7)
memory usage: 130.3+ KB


In [211]:
from sklearn.decomposition import PCA

pca = PCA(n_components= 3, random_state=42)

pca.fit(poke_num)
output = pd.DataFrame(pca.transform(poke_num), index=poke_num.index)

out=pd.merge(output, pokemon_forms, left_index=True, right_index=True)
pca.explained_variance_ratio_

array([0.61636408, 0.22512297, 0.05364436])

In [212]:
out

,0,1,2,pokemonid,formid,formname,gen,release,type1,type2,...,egg-cycles,friendship,growth-rate,ev-yield,hp,attack,defense,spatk,spdef,speed
bulbasaur,-93.468663,2.627523,-94.006256,bulbasaur,None,None,1,red-blue,grass,poison,...,20.0,50.0,medium slow,{'spatk': 1},45,49,49,65,65,45
ivysaur,-47.667306,-46.337298,-42.995597,ivysaur,None,None,1,red-blue,grass,poison,...,20.0,50.0,medium slow,"{'spatk': 1, 'spdef': 1}",60,62,63,80,80,60
venusaur,74.147866,-62.676457,13.529958,venusaur,None,None,1,red-blue,grass,poison,...,20.0,50.0,medium slow,"{'spatk': 2, 'spdef': 1}",80,82,83,100,100,80
venusaur-mega,149.176415,-66.811896,40.014437,venusaur,mega,Mega Venusaur,6,x-y,grass,poison,...,20.0,50.0,medium slow,"{'spatk': 2, 'spdef': 1}",80,100,123,122,120,80
charmander,-94.810680,4.540931,-95.037971,charmander,None,None,1,red-blue,fire,None,...,20.0,50.0,medium slow,{'speed': 1},39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chi-yu,36.728188,-174.441849,29.829129,chi-yu,None,None,9,scarlet-violet,dark,fire,...,50.0,0.0,slow,{'spatk': 3},55,80,80,135,120,100
roaring-moon,348.962958,35.951547,5.485042,roaring-moon,None,None,9,scarlet-violet,dragon,dark,...,50.0,0.0,slow,{'attack': 3},105,139,71,55,101,119
iron-valiant,67.704832,-160.555845,33.834563,iron-valiant,None,None,9,scarlet-violet,fairy,fighting,...,50.0,0.0,slow,{'attack': 3},74,130,90,120,60,116
koraidon,312.642599,-42.035732,33.699565,koraidon,None,None,9,scarlet-violet,fighting,dragon,...,50.0,0.0,slow,{'attack': 3},100,135,115,85,100,135


In [213]:
import plotly.express as px

px.scatter_3d(out, x=0,y=1,z=2,symbol="growth-rate", color="type1")

In [214]:
from sklearn.manifold import TSNE
pca_X  = PCA(n_components=10)
pca3 = pca.fit_transform(poke_num)
tsne = TSNE()

proj = pd.merge(pd.DataFrame(tsne.fit_transform(pca3), index=poke_num.index), pokemon_forms, left_index=True, right_index=True)
fig = px.scatter(proj, x=0, y = 1, color = proj["release"], hover_name=proj.index, color_discrete_sequence=px.colors.qualitative.Alphabet).show()

In [215]:
proj

,0,1,pokemonid,formid,formname,gen,release,type1,type2,species,...,egg-cycles,friendship,growth-rate,ev-yield,hp,attack,defense,spatk,spdef,speed
bulbasaur,-34.326740,-18.434347,bulbasaur,None,None,1,red-blue,grass,poison,Seed,...,20.0,50.0,medium slow,{'spatk': 1},45,49,49,65,65,45
ivysaur,-11.043213,-23.263420,ivysaur,None,None,1,red-blue,grass,poison,Seed,...,20.0,50.0,medium slow,"{'spatk': 1, 'spdef': 1}",60,62,63,80,80,60
venusaur,24.944283,-8.961027,venusaur,None,None,1,red-blue,grass,poison,Seed,...,20.0,50.0,medium slow,"{'spatk': 2, 'spdef': 1}",80,82,83,100,100,80
venusaur-mega,31.185814,-2.841331,venusaur,mega,Mega Venusaur,6,x-y,grass,poison,Seed,...,20.0,50.0,medium slow,"{'spatk': 2, 'spdef': 1}",80,100,123,122,120,80
charmander,-34.363750,-17.723093,charmander,None,None,1,red-blue,fire,None,Lizard,...,20.0,50.0,medium slow,{'speed': 1},39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chi-yu,38.186382,-18.771109,chi-yu,None,None,9,scarlet-violet,dark,fire,Ruinous,...,50.0,0.0,slow,{'spatk': 3},55,80,80,135,120,100
roaring-moon,30.923147,11.053571,roaring-moon,None,None,9,scarlet-violet,dragon,dark,Paradox,...,50.0,0.0,slow,{'attack': 3},105,139,71,55,101,119
iron-valiant,37.802002,-15.426898,iron-valiant,None,None,9,scarlet-violet,fairy,fighting,Paradox,...,50.0,0.0,slow,{'attack': 3},74,130,90,120,60,116
koraidon,32.434929,9.125082,koraidon,None,None,9,scarlet-violet,fighting,dragon,Paradox,...,50.0,0.0,slow,{'attack': 3},100,135,115,85,100,135


In [216]:
stats

,hp,attack,defense,spatk,spdef,speed
bulbasaur,45,49,49,65,65,45
ivysaur,60,62,63,80,80,60
venusaur,80,82,83,100,100,80
venusaur-mega,80,100,123,122,120,80
charmander,39,52,43,60,50,65
...,...,...,...,...,...,...
iron-crown,90,72,100,122,108,98
terapagos-normal,90,65,85,65,85,60
terapagos-terastal,95,95,110,105,110,85
terapagos-stellar,160,105,110,130,110,85


In [217]:

poke_f : pd.DataFrame= pokemon_forms[cols]
poke_f = poke_f.drop(columns=["release", "gender"])
poke_f = poke_f.merge(stats, left_index=True, right_index=True)
poke_types = pd.get_dummies(poke_f[['type1', 'type2']], prefix='', prefix_sep='')
poke_f.merge(poke_types, left_index=True, right_index=True)
poke_f = poke_f.drop(columns=['type1', 'type2'])
poke_f

,gen,height,weight,catch-rate,base-exp,egg-cycles,friendship,growth-rate,hp,attack,defense,spatk,spdef,speed
bulbasaur,1,0.7,6.9,45.0,64.0,20.0,50.0,medium slow,45,49,49,65,65,45
ivysaur,1,1.0,13.0,45.0,142.0,20.0,50.0,medium slow,60,62,63,80,80,60
venusaur,1,2.0,100.0,45.0,236.0,20.0,50.0,medium slow,80,82,83,100,100,80
venusaur-mega,6,2.4,155.5,45.0,281.0,20.0,50.0,medium slow,80,100,123,122,120,80
charmander,1,0.6,8.5,45.0,62.0,20.0,50.0,medium slow,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iron-crown,9,1.6,156.0,10.0,NaN,NaN,NaN,slow,90,72,100,122,108,98
terapagos-normal,9,0.2,6.5,255.0,NaN,NaN,NaN,slow,90,65,85,65,85,60
terapagos-terastal,9,0.3,16.0,255.0,NaN,NaN,NaN,slow,95,95,110,105,110,85
terapagos-stellar,9,1.7,77.0,255.0,NaN,NaN,NaN,slow,160,105,110,130,110,85


In [218]:
stats.columns

Index(['hp', 'attack', 'defense', 'spatk', 'spdef', 'speed'], dtype='object')

In [219]:
pokemon_forms.columns

Index(['pokemonid', 'formid', 'formname', 'gen', 'release', 'type1', 'type2',
       'species', 'height', 'weight', 'gender', 'catch-rate', 'base-exp',
       'egg-cycles', 'friendship', 'growth-rate', 'ev-yield', 'hp', 'attack',
       'defense', 'spatk', 'spdef', 'speed'],
      dtype='object')

In [220]:

pca_type  = PCA(n_components=3, random_state=42)
pca_type.fit(poke_types)
pca_type_3  = pca_type.transform(poke_types)
px.scatter(pd.DataFrame(pca_type_3, index=poke_types.index), x=0, y=1, color=2, 
           hover_name=poke_types.index).show()
explained_variance = pca_type.explained_variance_ratio_
fig = px.bar(x=range(1, len(explained_variance) + 1), y=explained_variance, labels={'x': 'Principal Component', 'y': 'Explained Variance Ratio'})
#fig.show()

In [221]:
pca_type.explained_variance_ratio_
explained_variance_cumsum = np.cumsum(pca_type.explained_variance_ratio_)
fig = px.line(x=range(1, len(explained_variance_cumsum) + 1), y=explained_variance_cumsum, labels={'x': 'Principal Component', 'y': 'Cumulated Explained Variance Ratio'})
fig.show()

In [222]:
tsne = TSNE()

proj = pd.merge(pd.DataFrame(tsne.fit_transform(pca_type_3), index=poke_types.index), pokemon_forms, left_index=True, right_index=True)
fig = px.scatter(proj, x=0, y = 1, color = proj["release"], hover_name=proj.index, color_discrete_sequence=px.colors.qualitative.Alphabet).show()

In [224]:
pca_type  = PCA(n_components=3)
pca_type_3 = pca_type.fit_transform(poke_types)
px.scatter(pd.DataFrame(pca_type_3, index=poke_types.index), x=0, y=1, color=2, hover_name=poke_types.index, hover_data=pca_types).show()
tsne = TSNE()

proj = pd.merge(pd.DataFrame(tsne.fit_transform(pca_type_3), index=poke_types.index), pokemon_forms, left_index=True, right_index=True)
#fig = px.scatter(proj, x=0, y = 1, color = proj["release"], hover_name=proj.index, color_discrete_sequence=px.colors.qualitative.Alphabet).show()

NameError: name 'pca_types' is not defined

In [ ]:
type1_set = set(pokemon_forms['type1'])
type2_set = set(pokemon_forms['type2'])

In [ ]:
type_to_pokemon = {}

for poke_type in type1_set:
    type_to_pokemon[poke_type] = pokemon_forms[pokemon_forms['type1'] == poke_type]['pokemonid'].tolist() + \
                                 pokemon_forms[pokemon_forms['type2'] == poke_type]['pokemonid'].tolist()

type_to_pokemon

{'water': ['squirtle',
  'wartortle',
  'blastoise',
  'blastoise',
  'psyduck',
  'golduck',
  'poliwag',
  'poliwhirl',
  'poliwrath',
  'tentacool',
  'tentacruel',
  'slowpoke',
  'slowbro',
  'slowbro',
  'seel',
  'dewgong',
  'shellder',
  'cloyster',
  'krabby',
  'kingler',
  'horsea',
  'seadra',
  'goldeen',
  'seaking',
  'staryu',
  'starmie',
  'magikarp',
  'gyarados',
  'gyarados',
  'lapras',
  'vaporeon',
  'totodile',
  'croconaw',
  'feraligatr',
  'chinchou',
  'lanturn',
  'marill',
  'azumarill',
  'politoed',
  'wooper',
  'quagsire',
  'slowking',
  'qwilfish',
  'corsola',
  'remoraid',
  'octillery',
  'mantine',
  'kingdra',
  'suicune',
  'mudkip',
  'marshtomp',
  'swampert',
  'swampert',
  'lotad',
  'lombre',
  'ludicolo',
  'wingull',
  'pelipper',
  'carvanha',
  'sharpedo',
  'sharpedo',
  'wailmer',
  'wailord',
  'barboach',
  'whiscash',
  'corphish',
  'crawdaunt',
  'feebas',
  'milotic',
  'castform',
  'clamperl',
  'huntail',
  'gorebyss',
  

In [ ]:


# Create a DataFrame with types as columns and rows
type_matrix = pd.DataFrame(0, index=list(type1_set), columns=list(type1_set))

# Populate the DataFrame with counts of unique Pokémon for each type combination
for type1, pokemons1 in type_to_pokemon.items():
    for type2, pokemons2 in type_to_pokemon.items():
        unique_elements = set(pokemons1 + pokemons2)
        duplicate_count = len(pokemons1) + len(pokemons2) - len(unique_elements)
        #print(f"Duplicate elements count: {duplicate_count}")
        #print(f"Unique elements: {unique_elements} between {type1} and {type2}")
        type_matrix.loc[type1, type2] = duplicate_count

#print(type_matrix)

import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
    z=type_matrix.values,
    x=type_matrix.columns,
    y=type_matrix.index,
    colorscale='Viridis',
    hoverongaps=False))

fig.update_layout(
    title='Heatmap of Pokémon Type Combinations',
    xaxis_nticks=36,
    yaxis_nticks=36)

fig.show()


# Classificateur non supervisé


In [ ]:
pokemon_forms["growth-rate"].unique()
pok_growth_rate = {'medium slow' : 2, 'medium fast' : 4, 'fast' : 5, 'slow' : 1, 'fluctuating' : 3,
       'erratic': 0}

In [ ]:
poke_class = pokemon_forms
col_del = ["pokemonid", "formid", "formname", "release", "ev-yield" ]
poke_class.drop(columns=col_del, inplace=True)


In [ ]:
poke_class["growth-rate"] = poke_class["growth-rate"].map(pok_growth_rate)
poke_class

,gen,type1,type2,species,height,weight,gender,catch-rate,base-exp,egg-cycles,friendship,growth-rate,hp,attack,defense,spatk,spdef,speed
bulbasaur,1,grass,poison,Seed,0.7,6.9,1:7,45.0,64.0,20.0,50.0,2,45,49,49,65,65,45
ivysaur,1,grass,poison,Seed,1.0,13.0,1:7,45.0,142.0,20.0,50.0,2,60,62,63,80,80,60
venusaur,1,grass,poison,Seed,2.0,100.0,1:7,45.0,236.0,20.0,50.0,2,80,82,83,100,100,80
venusaur-mega,6,grass,poison,Seed,2.4,155.5,1:7,45.0,281.0,20.0,50.0,2,80,100,123,122,120,80
charmander,1,fire,None,Lizard,0.6,8.5,1:7,45.0,62.0,20.0,50.0,2,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iron-crown,9,steel,psychic,Paradox,1.6,156.0,genderless,10.0,NaN,NaN,NaN,1,90,72,100,122,108,98
terapagos-normal,9,normal,None,Tera,0.2,6.5,4:4,255.0,NaN,NaN,NaN,1,90,65,85,65,85,60
terapagos-terastal,9,normal,None,Tera,0.3,16.0,4:4,255.0,NaN,NaN,NaN,1,95,95,110,105,110,85
terapagos-stellar,9,normal,None,Tera,1.7,77.0,4:4,255.0,NaN,NaN,NaN,1,160,105,110,130,110,85


In [ ]:
poke_class = poke_class.drop(columns=["type1", "type2"])
poke_class.merge(poketypes,left_index=True, right_index=True )

NameError: name 'poketypes' is not defined